In [1]:
%%bash

# install the ncl language in the conda environment, if it's not already installed.
if  which ncl >/dev/null; then
    echo NCL is already installed!
else
    conda install -y ncl
fi

Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs: 
    - ncl


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.3.9           |           py36_0         149 KB  conda-forge
    xorg-libxaw-1.0.13         |    h14c3975_1002         373 KB  conda-forge
    xorg-libxmu-1.1.3          |       h516909a_0          90 KB  conda-forge
    esmf-7.1.0r                |       ha678704_2        19.1 MB  conda-forge
    openssl-1.1.1b             |       h14c3975_1         4.0 MB  conda-forge
    hdfeos2-2.20               |    h64bfcee_1000         236 KB  conda-forge
    ncl-6.5.0                  |blas_openblashd40de8d_1        59.6 MB  conda-forge
    xorg-makedepend-1.0.6      |       he1b5a44_1          25 KB  conda-forge
    xorg-imake-1.0.7           |                0          23 KB  conda-forge
    hdfeos5-5



==> WARNING: A newer version of conda exists. <==
  current version: 4.5.4
  latest version: 4.6.8

Please update conda by running

    $ conda update -n base conda


certifi-2019.3.9     |  149 KB | ########## | 100% 
xorg-libxaw-1.0.13   |  373 KB | ########## | 100% 
xorg-libxmu-1.1.3    |   90 KB | ########## | 100% 
esmf-7.1.0r          | 19.1 MB | ########## | 100% 
openssl-1.1.1b       |  4.0 MB | ########## | 100% 
hdfeos2-2.20         |  236 KB | ########## | 100% 
ncl-6.5.0            | 59.6 MB | ########## | 100% 
xorg-makedepend-1.0. |   25 KB | ########## | 100% 
xorg-imake-1.0.7     |   23 KB | ########## | 100% 
hdfeos5-5.1.16       |  532 KB | ########## | 100% 
mpi-1.0              |    4 KB | ########## | 100% 
jasper-1.900.1       |  286 KB | ########## | 100% 
mpich-3.2.1          |  5.3 MB | ########## | 100% 
netcdf-fortran-4.4.4 |  885 KB | ########## | 100% 
udunits2-2.2.27.6    |  152 KB | ########## | 100% 
ca-certificates-2019 |  146 KB | ########## | 100% 

Create the regridding weight file.

In [ ]:
!ncdump -h input_files

In [ ]:
%%bash

FILE=`find input_files -name '*.grb' | head -1`
echo $FILE

ncl 'interp_opt="bilinear"' \
'srcGridName="'$FILE'"' \
'dstGridName="DOMAIN/geo_em.d01.nc"' \
NLDAS2WRFHydro_generate_weights.ncl


Regrid all of the NLDAS files in the `input_files` directory using the NLDAS2WRFHydro_regrid.ncl script. This script takes NLDAS data and a weight file as inputs and outputs regridded data.

In [ ]:
%%bash

ncl 'srcFileName="NLDAS_FORA0125_H.*"'\
 'dstGridName="DOMAIN/geo_em.d01.nc"' \
NLDAS2WRFHydro_regrid.ncl


Clean the our directory by (1) removing the raw NLDAS data that we downloaded from EarthData and (2) rename the default `output_files` directory to `FORCING`.

In [ ]:
!rm -rf input_files
!mv output_files FORCING
!ls FORCING

The next notebook demonstrates how these `FORCING` data can be used to run the WRF-Hydro model. However, execution of the model must be completed on your local machine, so first we'll save our data to HydroShare. Replace the `<abstract>`, `<title>`, and `<keyword>`, placeholders below:

In [ ]:
from utilities import hydroshare
hs = hydroshare.hydroshare()

# compress the DOMAIN and FORCING directories
!tar -czf wrfhydro-data.tar.gz DOMAIN FORCING

# lets save this content as a new resource in HydroShare
abstract = '<abstract>'
title = '<title>'    
keywords = ['<keyword>', '<keyword>2']         

# create the new resource
resource_id = hs.createHydroShareResource(abstract, 
                                          title, 
                                          keywords=keywords, 
                                          resource_type='compositeresource', 
                                          content_files=['wrfhydro-data.tar.gz',
                                                         'WRFHydro-Domain.ipynb',
                                                         'WRFHydro-Forcing.ipynb,
                                                         'WRFHydro-Simulation.ipynb'], 
                                          public=False)